# Projeto 2 - Ciência dos Dados

Nome: Gabriela Yukari Mitu

Nome: Gabriella Escobar Cukier

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [ ]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [ ]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***@gabriela_ymitu***

In [ ]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @gabriela_ymitu

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [ ]:
#Produto escolhido:
produto = 'Android'

#Quantidade mínima de mensagens capturadas:
n = 1000
#Quantidade mínima de mensagens para a base de treinamento:
t = 600

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    if msg.full_text.lower()[0] != 'r' and msg.full_text.lower()[1] != 't':
        msgs.append(msg.full_text.lower())
        i += 1
    if i > n:
        break

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [ ]:
dft=pd.read_excel('Android.xlsx','Treinamento',sep=',')
dfc=pd.read_excel('Android.xlsx', 'Teste',sep=',')

In [ ]:
dft

In [ ]:
import re
#limpo = re.sub("\.{1,}", "?", "dft")

In [ ]:
lista1=['  ','   ',':','(',')','`','[',']','.','/','" ',"'"]
lista2=[' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ']
for i in range(len(lista1)):
    dft['Treinamento']=dft.Treinamento.str.replace(lista1[i], lista2[i])
    dfc['Teste']=dfc.Teste.str.replace(lista1[i], lista2[i])
dft['Treinamento']=dft.Treinamento.str.lower()
dfc['Teste']=dfc.Teste.str.lower()

In [ ]:
dft['Treinamento']=dft['Treinamento'].str.split()
dfc['Teste']=dfc['Teste'].str.split()
dft['Treinamento'].head()

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [ ]:
dftLen=len(dft)

dftIrre=dft[dft['Relevância']==0]
dftIrreLen=len(dftIrre)

dftNeutro=dft[dft['Relevância']==1]
dftNeutroLen=len(dftNeutro)

dftRelev=dft[dft['Relevância']==2]
dftRelevLen=len(dftRelev)

dftMtRelev=dft[dft['Relevância']==3]
dftMtRelevLen=len(dftMtRelev)

In [ ]:
# Contagem de palavras dado sua categoria
#Irrelevante
dftIrreProb=dftRelev["Treinamento"].sum()
dftIrreProb=pd.Series(dftIrreProb)
dftIrreProb=dftIrreProb.value_counts()

#Neutro
dftNeutroProb=dftNeutro["Treinamento"].sum()
dftNeutroProb=pd.Series(dftNeutroProb)
dftNeutroProb=dftNeutroProb.value_counts()

#Relevante
dftRelevProb=dftRelev["Treinamento"].sum()
dftRelevProb=pd.Series(dftRelevProb)
dftRelevProb=dftRelevProb.value_counts()

#Muito relevante
dftMtRelevProb=dftMtRelev["Treinamento"].sum()
dftMtRelevProb=pd.Series(dftMtRelevProb)
dftMtRelevProb=dftMtRelevProb.value_counts()

#Total
dftTotal=dft.Treinamento.sum()
dftTotal=pd.Series(dftTotal)
dftTotal=dftTotal.value_counts()

In [ ]:
dftPalavras=dft.Treinamento.sum()
dftPalavras=pd.Series(dftPalavras)
dftPalavras=len(dftPalavras)

#Check de palavras
check=dft.Treinamento.sum()
check=pd.Series(check)

In [ ]:
#Números totais de palavras em cada categoria, mas agora em variáveis
#Irrelevante
IrrePalavras=dftIrre["Treinamento"].sum()
IrrePalavras=pd.Series(IrrePalavras)
IrrePalavras=len(IrrePalavras)

#Neutro
NeutroPalavras=dftNeutro["Treinamento"].sum()
NeutroPalavras=pd.Series(NeutroPalavras)
NeutroPalavras=len(NeutroPalavras)

#Relevante
RelevPalavras=dftRelev["Treinamento"].sum()
RelevPalavras=pd.Series(RelevPalavras)
RelevPalavras=len(RelevPalavras)

#Muito Relevante
MtRelevPalavras=dftMtRelev["Treinamento"].sum()
MtRelevPalavras=pd.Series(MtRelevPalavras)
MtRelevPalavras=len(MtRelevPalavras)


In [ ]:
# Probabilidades base
ProbIrre=IrrePalavras/(dftPalavras)
ProbNeutro=NeutroPalavras/(dftPalavras)
ProbRelev=RelevPalavras/(dftPalavras)
ProbMtRelev=MtRelevPalavras/(dftPalavras)

print('A probablidade de ser Irrelevante é: {0:.3f} %'.format(ProbIrre*100))
print('A probablidade de ser Neutro é: {0:.3f} %'.format(ProbNeutro*100))
print('A probablidade de ser Relevante é: {0:.3f} %'.format(ProbRelev*100))
print('A probablidade de ser Irrelevante  é: {0:.3f} %'.format(ProbMtRelev*100))

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [ ]:
___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**